In [143]:
from __future__ import division
import json
import re, collections
import numpy as np
import os
base_dir=os.path.expanduser('~')
import datetime
import psycopg2
import json

#connect
#conn = psycopg2.connect("dbname=emoji_db user=postgres password=darkmatter")
conn = psycopg2.connect("dbname=qa")
#on AWS: conn = psycopg2.connect("host=localhost port=5432 dbname=emoji_db user=postgres password=darkmatter")
cur = conn.cursor()

In [144]:
#training table, initialize
cur.execute("CREATE TABLE training (id serial PRIMARY KEY,\
    question text,\
    qestion_type text,\
    qestion_type_bow text,\
    qestion_type_human text,\
    qa_id integer,\
    data_corr_yn float,\
    data_corr_oe float,\
    bow_corr_yn float,\
    bow_corr_oe float,\
    count_yn integer,\
    count_oe integer,\
    name text,\
    time timestamp\
    );")
conn.commit() #submit change to db

In [ ]:
#types
# open-ended
# yes/no

<h1> Insert into table </h1>

In [191]:
question='can you play thrash or speed metal on this amp'
qestion_type='open-ended'
qestion_type_bow= 'yes/no'
qestion_type_human= 'yes/no'
qa_id=1
data_corr_yn=0
data_corr_oe=0
bow_corr_yn=1
bow_corr_oe=0
count_yn=1
count_oe=0
name='test'
sub_time=datetime.datetime.now()

cur.execute("INSERT INTO training (\
question,\
qestion_type,\
qestion_type_bow,\
qestion_type_human,\
qa_id,\
data_corr_yn,\
data_corr_oe,\
bow_corr_yn,\
bow_corr_oe,\
count_yn,\
count_oe,\
name,\
time\
)\
VALUES (\
%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s\
)",(\
question,\
qestion_type,\
qestion_type_bow,\
qestion_type_human,\
qa_id,\
data_corr_yn,\
data_corr_oe,\
bow_corr_yn,\
bow_corr_oe,\
count_yn,\
count_oe,\
name,\
sub_time\
))
conn.commit() #submit change to db

In [196]:
cur.execute("SELECT name, count(*) from training group by name order by count DESC;")
result=cur.fetchall()

In [197]:
result

[('Attila', 2L), ('test', 1L)]

In [185]:
names=result[0]
counts=[int(val) for val in result[1]]

TypeError: 'long' object is not iterable

In [198]:
{"values":[{"rank":rank+1,"value":int(count),"label":name} for rank,(name,count) in enumerate(result)],"key": "Serie 1"}

{'key': 'Serie 1',
 'values': [{'label': 'Attila', 'rank': 1, 'value': 2},
  {'label': 'test', 'rank': 2, 'value': 1}]}

In [35]:
cur.execute("SELECT * from training order by id DESC limit 1;")

In [36]:
result=cur.fetchall()[0]

In [37]:
result

(1,
 'can you play thrash or speed metal on this amp',
 'open-ended',
 'yes/no',
 'yes/no',
 1,
 0.0,
 0.0,
 1.0,
 0.0,
 1,
 0)

In [52]:
yn=[1,2]
oe=[3,4]
label=['Data','Human']
key=['Yes/No','Data']

In [55]:
[{"values":[{"y":yn[0]*100,"x":label[0]},{"y":yn[1]*100,"x":label[1]}],"key":key[0],"yAxis":"1"},\
 {"values":[{"y":oe[0]*100,"x":label[0]},{"y":oe[1]*100,"x":label[1]}],"key":key[1],"yAxis":"1"}]

[{'key': 'Yes/No',
  'values': [{'x': 'Data', 'y': 100}, {'x': 'Human', 'y': 200}],
  'yAxis': '1'},
 {'key': 'Data',
  'values': [{'x': 'Data', 'y': 300}, {'x': 'Human', 'y': 400}],
  'yAxis': '1'}]

<h1> Test sentence model and combine BoW with QModel from word2vec </h1>

In [56]:
import re
from gensim import corpora, models, similarities

bag_of_words_yn='is,will,may,might,does,do,can,could,must,should,are,would,did'.split(',')
QmodelB=models.Word2Vec.load('QmodelB_update')

<h3> process the sentence </h3>

In [131]:
def process_line(sentence):
    #step 1, split
    sentences=re.split(r'[;:!?.-]\s*', sentence)
    result= [re.findall("[0-9a-z']+", sent.lower()) for sent in sentences if \
           re.findall("[0-9a-z']+", sent.lower())!=[]]
    if result==[]:
        result=['']
    #return first sentence and first word
    return ' '.join(result[0])+'?',result[0][0]
stoplist = set('for a of the and to in rt'.split())

<h3> Given a word check if it is similar enough to bag of words </h3>

In [136]:
sentence,first_word = process_line('can I do 8 of that?')
#first_word=sentence[0].lower()

try:
    is_in_bag=({first_word}|{item[0] for item in QmodelB.most_similar(first_word)})&set(bag_of_words_yn)!=set()
except KeyError:
    is_in_bag=False
is_in_bag

True

In [127]:
' '.join(sentence)+'?'

'can i do 8 of that?'

In [ ]:
if sentence.split()[0].lower() in bag_of_words_yn:
    qestion_type_bow='yes/no'
else:
    qestion_type_bow='open-ended'

In [138]:
conn.close()

<h1> bag of words </h1>

In [ ]:
self.bag_of_words_yn='is,will,wil,may,might,does,dose,doe,dos,do,can,could,must,should,are,would,do,did'.split(',')
